In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist =input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
n_inputs = 28#输入一行，每行28个数据
max_time = 28#28个序列
lstm_size = 100#隐藏单元,相当100个基础单元
n_classes = 10#10个分类
batch_size = 50#每批次50个样本
n_batch = mnist.train.num_examples // batch_size

In [4]:
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

In [5]:
weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
biases = tf.Variable(tf.constant(0.1,shape=[n_classes]))

In [6]:
def RNN(X,weights,biases):
    inputs = tf.reshape(X,[-1,max_time,n_inputs])
    #定义最基本的单元
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)#传入100个block
    #final_state[0]是cell state
    #final_state[1]是hidden state
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases)
    return results
    

In [7]:
prediction = RNN(x,weights,biases)


In [8]:
## 交叉熵代价函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
## 使用 AdamOptimizer进行优化
#####不同的optimizer造成的结果一般千差万别
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
## 结果存放在一个布尔列表中
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1)) ## argmax返回一维张量中最大的值所在的位置
## 求准确率
accuracy= tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [10]:
init = tf.global_variables_initializer()

In [15]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs,batch_ys =mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
       
        train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels})
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print ("Iter" + str(epoch)+" ,Testing Accuracy" +str(acc)+'training acc='+str(train_acc))
    

Iter0 ,Testing Accuracy0.7446training acc=0.7320182
Iter1 ,Testing Accuracy0.8312training acc=0.82745457
Iter2 ,Testing Accuracy0.8787training acc=0.88045454
Iter3 ,Testing Accuracy0.9051training acc=0.90672725
Iter4 ,Testing Accuracy0.9127training acc=0.91663635
Iter5 ,Testing Accuracy0.9242training acc=0.92565453
